In [ ]:
import pandas as pd
import numpy as np
from dis import dis
import math
import geopandas as gpd
import pandas as pd
import numpy
from shapely import wkt
from shapely import wkb
from shapely.geometry import Point
from geopandas import GeoDataFrame

modified = pd.read_csv('../data/modified_data.csv')
modified_test = pd.read_csv('../data/modified_test_data.csv')

# For train set
geometry_train = [Point(xy) for xy in zip(modified.lat, modified.lon)]
#modified = modified.drop(['lon', 'lat'], axis=1)
gdf_train = GeoDataFrame(modified, crs="EPSG:4326", geometry=geometry_train)

# For test set
geometry_test = [Point(xy) for xy in zip(modified_test.lat, modified_test.lon)]
#modified = modified.drop(['lon', 'lat'], axis=1)
gdf_test = GeoDataFrame(modified_test, crs="EPSG:4326", geometry=geometry_test)

#modified['geometry'] = [Point(xy) for xy in zip(modified.lat, modified.lon)]
#modified[]
#modified['geometry'] = modified['geometry'].replace(' POINT ', 'POINT')


grunnkrets_stripped = pd.read_csv('../data/grunnkrets_norway_stripped.csv')
busstops = pd.read_csv('../data/busstops_norway.csv')
bs = busstops

df_train = gdf_train
df_test = gdf_test


# def dist(lat, lon, df): 
#     lat1 = abs(df['lat'] - lat)
#     lon1 = abs(df['lon'] - lon)
#     eucdist = np.sqrt(lon1**2 + lat1**2)
#     print(type(eucdist))
#     return eucdist


In [ ]:
# Calculate distance between points in meters, https://math.stackexchange.com/questions/29157/how-do-i-convert-the-distance-between-two-lat-long-points-into-feet-meters
def dist_to_all_km(lat, lon, df):

    # coordinates in radians
    lat1 = lat*math.pi/180
    lon1 = lon*math.pi/180
    lat2 = df['lat']*math.pi/180 # go through whole lat column
    lon2 = df['lon']*math.pi/180 # go through whole lon column

    # store original coordinates in new dataframe
    distances = pd.DataFrame()
    distances['lat'] = df['lat'].copy()
    distances['lon'] = df['lon'].copy()

    # calculate cartesian coordinates
    R = 6371 # Earth radius in km
    df['x'] = R*np.cos(lat2)*np.cos(lon2)
    df['y'] = R*np.cos(lat2)*np.sin(lon2)
    df['z'] = R*np.sin(lat2)
    x1 = R*np.cos(lat1)*np.cos(lon1)
    y1 = R*np.cos(lat1)*np.sin(lon1)
    z1 = R*np.sin(lat1)

    # calculate distance, store as new column in the distances dataframe
    distances['dist'] = np.sqrt(np.square(df['x']-x1)+np.square(df['y']-y1)+np.square(df['z']-z1))

    return distances['dist'].squeeze()


In [ ]:
# import geopandas as gpd
# import shapely.geometry
# from shapely.geometry import Point
# import pandas as pd
# def dist_to_all_meter(lat, lon, data):
#     gdb = gpd.GeoDataFrame(
#         geometry=gpd.GeoSeries.from_wkt(
#             data["geometry"],
#             crs="epsg:4386",
#         )
#     )
#     point = Point(lat, lon)
#     gdb["distance"] = gdb.to_crs(gdb.estimate_utm_crs())["geometry"].distance(
#         gpd.GeoSeries([point for _ in range(len(gdb))], crs=gdb.crs).to_crs(
#             gdb.estimate_utm_crs()
#         )
#     ) 
#     return gdb
# dist_to_all_meter(50, 10, modified)



In [ ]:

def store_dist_lvl(distance, df): # enter distance in km
    #distance = 10
    df['num_stores_within_distance_and_same_lvl1'] = 0
    df['num_stores_within_distance_and_same_lvl2'] = 0
    df['num_stores_within_distance'] = 0
    num = 0 

    for index in range(len(df)):
        lat = df._get_value(index,'lat')
        lon = df._get_value(index,'lon')
        lvl1 = df._get_value(index, 'lv1')
        lvl2 = df._get_value(index, 'lv2')
        dist_to_all = dist_to_all_km(lat, lon, df)

        count_distance_and_lvl1 = 0 
        count_distance_and_lvl2 = 0 
        count_distance = 0 #only for distance
        iter = 0
        
        for number in dist_to_all:
            if number < distance:
                count_distance += 1
                if lvl1 == df._get_value(iter, 'lv1'):
                    count_distance_and_lvl1 += 1
                    if lvl2 == df._get_value(iter, 'lv2'):
                        count_distance_and_lvl2 += 1
            iter += 1

        df._set_value(index, 'num_stores_within_distance_and_same_lvl1', count_distance_and_lvl1-1)
        df._set_value(index, 'num_stores_within_distance_and_same_lvl2', count_distance_and_lvl2-1)
        df._set_value(index, 'num_stores_within_distance', count_distance-1)

        num+=1 #for runtime tracking only
        print(str(num) + "/12860")


In [ ]:
def num_busstops_wihin_distance_x(x, df): # enter x in km
    #busstops['geometry'] = gpd.GeoSeries.from_wkt(busstops['geometry'])
    busstops['lon'] = busstops.geometry.apply(lambda x: x.x)
    busstops['lat'] = busstops.geometry.apply(lambda x: x.y)
    df['num_busstops_within_distance'] = 0

    for index in range(len(df)):
        one_to_all = dist_to_all_km(df._get_value(index, 'lat'), df._get_value(index, 'lon'), busstops)
        one_to_all = one_to_all.to_frame()
        one_to_all.rename( columns={0 :'a'}, inplace=True)
        count = (one_to_all['a'] > x).sum()
        df._set_value(index, 'num_busstops_within_distance', count)
    




In [ ]:
store_dist_lvl(1, df_train)

In [ ]:
store_dist_lvl(1, df_test)

In [ ]:
#num_busstops_wihin_distance_x(1)

In [ ]:
df_train = df_train.drop(columns=df_train.columns[0], axis=1)
df_train = df_train.drop(['grunnkrets_id'], axis=1)
df_train = df_train.drop(['x'], axis=1)
df_train = df_train.drop(['y'], axis=1)
df_train = df_train.drop(['z'], axis=1)
df_train = df_train.drop(['lat'], axis=1)
df_train = df_train.drop(['lon'], axis=1)
df_train = df_train.drop(['geometry'], axis=1)

# For test
df_test = df_test.drop(columns=df_test.columns[0], axis=1)
df_test = df_test.drop(['grunnkrets_id'], axis=1)
df_test = df_test.drop(['x'], axis=1)
df_test = df_test.drop(['y'], axis=1)
df_test = df_test.drop(['z'], axis=1)
df_test = df_test.drop(['lat'], axis=1)
df_test = df_test.drop(['lon'], axis=1)
df_test = df_test.drop(['geometry'], axis=1)

col_at_end = ['revenue']
df_train = df_train[[c for c in df_train if c not in col_at_end] + [c for c in col_at_end if c in df_train]]

df_train.to_csv('train_set.csv', index=False)
df_test.to_csv('test_set.csv', index=False)
df_test.head()